# Neural Network Model Definition and Evaluation

This notebook defines, trains, and evaluates a neural network model to predict
daily bakery sales based on weather variables and Kieler Woche.

Model performance is compared against a baseline linear regression model.

## Imports and Setup

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


## Reproducibility

In [2]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


## Load Dataset


In [3]:
PROJECT_ROOT = Path("..")
DATA_PATH = PROJECT_ROOT / "merged_daily_sales_weather.csv"

df = pd.read_csv(DATA_PATH, parse_dates=["Datum"])
df.head()


,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,0
1,1307021,2013-07-02,1,159.793757,3.0,17.3125,10.0,NaN,0
2,1307031,2013-07-03,1,111.885594,7.0,21.0750,6.0,61.0,0
3,1307041,2013-07-04,1,168.864941,7.0,18.8500,7.0,20.0,0
4,1307051,2013-07-05,1,171.280754,5.0,19.9750,12.0,NaN,0


## Variable Selection


In [4]:
FEATURES = [
    "Bewoelkung",
    "Temperatur",
    "Windgeschwindigkeit",
    "Wettercode",
    "KielerWoche"
]

TARGET = "Umsatz"

df_model = df[FEATURES + [TARGET]].copy()


## Missing Value Handling

Missing values are imputed using the median of each feature.


In [5]:
df_model[FEATURES] = df_model[FEATURES].fillna(
    df_model[FEATURES].median()
)


## Train/Test Split


In [6]:
X = df_model[FEATURES].values
y = df_model[TARGET].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)


## Feature Scaling


In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Neural Network Architecture

A simple fully connected feedforward neural network is used.
Early stopping is applied to prevent overfitting.


In [8]:
model = Sequential([
    Dense(32, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(16, activation="relu"),
    Dense(1)
])

model.compile(
    optimizer="adam",
    loss="mse"
)

model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                192       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 737 (2.88 KB)
Trainable params: 737 (2.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Model Training


In [9]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/200

187/187 [==============================] - 1s 2ms/step - loss: 63688.5820 - val_loss: 56685.2227
Epoch 2/200
187/187 [==============================] - 0s 2ms/step - loss: 46095.0664 - val_loss: 29490.7227
Epoch 3/200
187/187 [==============================] - 0s 2ms/step - loss: 25011.8262 - val_loss: 21126.6738
Epoch 4/200
187/187 [==============================] - 0s 2ms/step - loss: 22590.3828 - val_loss: 20670.8281
Epoch 5/200
187/187 [==============================] - 0s 2ms/step - loss: 22319.6777 - val_loss: 20468.6719
Epoch 6/200
187/187 [==============================] - 0s 1ms/step - loss: 22244.1016 - val_loss: 20347.8359
Epoch 7/200
187/187 [==============================] - 0s 2ms/step - loss: 21875.8672 - val_loss: 20265.0488
Epoch 8/200
187/187 [==============================] - 0s 2ms/step - loss: 21941.2637 - val_loss: 20209.0723
Epoch 9/200
187/187 [==============================] - 0s 2ms/step - loss: 21634.8965 - val_loss: 20154.7520
Epoch 10/200
187/1

## Model Evaluation


In [10]:
y_pred = model.predict(X_test_scaled).flatten()

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

rmse, mae, r2


59/59 [==============================] - 0s 854us/step


C:\Users\Vaishali\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


(128.7679434244612, 103.69349541280755, 0.053774344234512106)

## Comparison with Baseline Model

Baseline performance (Linear Regression):
- RMSE ≈ 129.14
- MAE ≈ 104.27
- R² ≈ 0.05


In [11]:
comparison = pd.DataFrame({
    "Model": ["Baseline (Linear)", "Neural Network"],
    "RMSE": [129.14, rmse],
    "MAE": [104.27, mae],
    "R2": [0.048, r2]
})

comparison


,Model,RMSE,MAE,R2
0,Baseline (Linear),129.140000,104.270000,0.048000
1,Neural Network,128.767943,103.693495,0.053774
